<a href="https://colab.research.google.com/github/jooeun921/Big-Data-Analyst/blob/main/Part03_Section_04_scikit_learn_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Section 04 학습 : scikit-learn을 활용한 군집분석 수행
> 군집분석은 아직까지 시험에서 출제된 적이 없음.

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/USArrests.csv')
print(df.head(2))

In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_data = df.select_dtypes('number')
stdscaler = StandardScaler()

df_trans = pd.DataFrame(stdscaler.fit_transform(numeric_data), columns = numeric_data.columns)
print(df_trans.head())

#### K-평균 군집분석
장점 : 구현이 비교적 간단. 계산속도가 빠름   
단점 : 사전에 k값 지정 필요. 이상치에 민감

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 4, random_state = 0)
labels = kmeans.fit_predict(df_trans)
print(labels)

In [ ]:
df['cluster_label'] = labels

print(df.head(2))

#### 계층적 군집분석
AGNES와 DIANA로 구분할 수 있음.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

hk = AgglomerativeClustering(n_clusters = 4, linkage = "single")
hk.fit(df_trans)

In [ ]:
df['cluster_label2'] = hk.labels_

In [ ]:
df.head(3)

#### 외부 유효성 지표
대표적으로 Rand Index, Adjusted Rand Index가 있음.

Rand Index는 0 ~ 1 사이 값을 가지며, 1에 가까울수록 군집 알고리즘의 성능이 높음.

ARI(Adjusted Rand Index)는 -1 ~ 1 사이 값을 가지며, 1에 가까울수록 군집 알고리즘의 성능이 높음.

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
y_true = [1, 0, 1, 1, 1, 1, 2]
y_pred = [1, 1, 0, 0, 1, 1, 2]
conf_mat = confusion_matrix(y_true, y_pred, labels=[0, 1, 2])
print("\nConfusion Matrix:")
print(conf_mat)

In [ ]:
from sklearn.metrics import rand_score

ri = rand_score(y_true, y_pred)
print(f"Rand Index: {ri: .3f}")

In [ ]:
from sklearn.metrics import adjusted_rand_score

ari = adjusted_rand_score(y_true, y_pred)
print(f"Adjusted Rand Index: {ari: .3f}")

#### 내부 유효성 지표
정답이 주어지지 않은 상태에서 군집 자체의 품질을 판단하기 위해 설계된 지표.   

- 응집도 : 같은 군집 내 관측치들이 얼마나 모여있는지. 군집 내 분산이 작으면 군집의 성능이 높다.   
- 분리도 : 군집이 다른 군집과 잘 분리되었는지. 군집 간 거리가 멀면 성능이 높다.   

대표적인 지표에는 실루엣계수와 팔꿈치 방법이 있음.

In [ ]:
# 실루엣계수 : 응집도와 분리도를 동시에 고려. 평균 실루엣 계수가 가장 큰 k를 군집의 수로 정하게 됨.

from sklearn.metrics import silhouette_score

scores = []

for i in range(2, 10):
    fit_kmeans = KMeans(n_clusters = i, random_state = 0).fit(df_trans)
    score = silhouette_score(df_trans, fit_kmeans.labels_)
    scores.append(score)
    print("For n_clusters = {0}, the silhouette score is {1}".format(i, score))

In [ ]:
kmeans = KMeans(n_clusters = 2)
labels = kmeans.fit_predict(df_trans)

In [ ]:
# 팔꿈치 방법 : k에 다른 군집 내 총 변동을 계산하고, 그 변동이 급격히 줄어드는 지점(팔꿈치 지점)을 찾아 최적의 군집 수를 결정

kmeans.inertia_

In [ ]:
wss = []

for i in range(2, 10):
    fit_kmeans = KMeans(n_clusters = i, random_state = 0).fit(df_trans)
    wss.append(fit_kmeans.inertia_)
    print("For n_clusters = {0}, WSS = {1}".format(i, fit_kmeans.inertia_))